In [2]:
import pandas as pd
import openai
from keybert import KeyBERT
import langdetect
import time
import numpy as np
import requests
from PIL import Image
from io import BytesIO

from transformers import pipeline

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 200)

# model = KeyBERT('distilbert-base-nli-mean-tokens')

In [ ]:
with open("apikey_moritz.txt", "r") as f:
    api_key = f.read()

print(api_key)
openai.api_key = api_key

## Text: Finetuned GPT3 Model


In [ ]:
fine_tune_list = openai.FineTune.list()
fine_tuned_model = fine_tune_list['data'][0].fine_tuned_model
print(fine_tuned_model)

In [ ]:
new_prompt = "A Google Maps review about a steak house. ###"
answer = openai.Completion.create(
  model=fine_tuned_model,
  prompt=new_prompt,
  max_tokens=200, # Change amount of tokens for longer completion
  temperature=1,
  stop = "END"
)
print(answer['choices'][0]['text'])

In [ ]:
def gpt3_finetuned(row):
    catgeoryName = row['categoryName'].lower()
    prompt = "A Google Maps review about a " + catgeoryName + ". ###"
    print(prompt)
    answer = openai.Completion.create(
      model=fine_tuned_model,
      prompt=prompt,
      max_tokens=200, # Change amount of tokens for longer completion
      temperature=1, 
      stop = "END"
    )
    return answer['choices'][0]['text']

# def gpt3_5_turbo(row):
#     prompt = "Write a Google Maps review about a " + row['categoryName'] + ". In Style of a Google user that writes a review about a place."
#     role = "In stlye of a google user that writes a review about a place."
#     prompt = "Write a Google Maps review about a " + row['categoryName']+"."
#     print(prompt)
#     return chatGPT(api_key, role).question(prompt)

#### Outdated: Nur für Test:  gpt3_5_turbo vs gpt3_finetuned 

In [ ]:
# ## df_test.to_csv('test_df_showcase20gpt.csv')
# # print("Done with gpt3_finetuned_v1 text generation.")

# df_test = pd.read_csv('test_df_showcase.csv')
# df_test['gpt3_finetuned_v1'] = df_test.apply(gpt3_finetuned, axis=1)
# df_test.to_csv('test_df_showcase.csv')
# print("Done with gpt3.5_turbo text generation.")

# print(df_test)
# print(df_test.shape)

In [ ]:
# df_test = pd.read_csv('test_df_showcase.csv')
# df_test['gpt3.5_turbo'] = df_test['gpt3.5_turbo'].astype(str)

# for index, row in df_test.iterrows():
#     print(row['title'])
#     print(row['categoryName'])
#     print("ORG:" + row['original_text'])
#     print("GPT:" + row['gpt3_finetuned_v1'])
#     print("GPT3.5:" + row['gpt3.5_turbo'])
#     print("__________________________________________________________________________________________")

### Jetzt tatsächliche Daten augmentiert / generiert. Dazu einen neuen Dataframe erstellen...

Später wird der dann bemerged.

In [ ]:
# df_true = pd.read_csv('base_keywords_sentiment_reduced.csv')
# df_finetune = pd.read_csv('dataset_ft_v3_en_mulitmod_pc.csv.csv')

# # print(df_true['categoryName'].value_counts())
# # print(df_finetune['categoryName'].value_counts())
# print(f'True:' , df_true.shape)

# df_fake = pd.DataFrame(columns=df_true.columns)
# df_fake['categoryName'] = df_true['categoryName']
# df_fake['label'] = 'FAKE'

# # just for reference. Can be delted later on. 
# df_fake['reviewId'] = df_true['reviewId']
# df_fake['title'] = df_true['title']
# df_fake['text'] = df_true['text']
# df_fake['placeId'] = df_true['placeId']

# df_fake.to_csv('fake_base.csv', index=False)

#### Ab hier neue finetunes ausprobieren und unter neuer Version speichern. 
 Wichtig! Spaltennamen ändern von gpt3_v3

In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v2331.csv')
df_fake["prompt_v4"] = "A Google Maps review about a " + df_fake["categoryName"] + ". ###"

for index, row in df_fake.iterrows():
    df_fake.at[index, 'gpt3_v4'] = gpt3_finetuned(row)
    print(df_fake.at[index, 'gpt3_v4'])
    print(index)
    if index % 50 == 0:
        df_fake.to_csv('fake_base_gpt3_v23314.csv', index=False)
        print("Wir warten ein paar Sekunden und hoffen, dass der API key deshalb nicht geblockt wird...")
        time.sleep(25)
        print("Weiter geht's...")

print("Done with gpt3 text generation.")
print(df_fake.shape)
print(df_fake.columns)

df_fake.to_csv("fake_base_gpt3_v23314.csv", index=False)

In [ ]:
# load df_fake and save in xlsx

df_fake = pd.read_csv('fake_base_gpt3_v23314.csv')
print(df_fake.shape)


# Image Generierung


Vorbereitung. Eine Sentiment-Analyse auf dem generierten Text durchführen. 

In [ ]:
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [ ]:
# do a sentiment analysis on the generated text gpt3_v2, gpt3_v3 gpt3_v3.1

df_fake = pd.read_csv('fake_base_gpt3_v23314.csv')
df_fake['sent_v2'] = np.nan
df_fake['sent_v3']= np.nan
df_fake['sent_v3.1'] = np.nan
df_fake['sent_v4'] = np.nan

df_fake['sent_v2'] = df_fake['gpt3_v2'].astype(str)
df_fake['sent_v3'] = df_fake['gpt3_v3'].astype(str)
df_fake['sent_v3.1'] = df_fake['gpt3_v3.1'].astype(str)
df_fake['sent_v4'] = df_fake['gpt3_v4'].astype(str)

for index, row in df_fake.iterrows():
    print("Processing row: ", index)
    try:
        df_fake.at[index, 'sent_v2'] = sentiment_task(row['gpt3_v2'])[0]['label']
        df_fake.at[index, 'sent_v3'] = sentiment_task(row['gpt3_v3'])[0]['label']
        df_fake.at[index, 'sent_v3.1'] = sentiment_task(row['gpt3_v3.1'])[0]['label']
        df_fake.at[index, 'sent_v4'] = sentiment_task(row['gpt3_v4'])[0]['label']
    except RuntimeError:
        print("Skipped due to messy data. Row: ", index)
        pass

df_fake.to_csv('fake_base_gpt3_v23314_sent.csv', index=False)
df_fake.to_excel('fake_base_gpt3_v23314_sent.xlsx', index=False)


## Image: Dall-E-2


In [ ]:
with open("apikey_papa.txt", "r") as f:
    openai.api_key = f.read()

In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v23314_sent.csv')
print(df_fake.shape)
print(df_fake.columns)
print(df_fake['categoryName'])

In [ ]:
def generate_dalle_prompt(row):
    genre = row["genre"].lower()
    categoryName = row["categoryName"].lower()
    sentiment = row["sent_v4"].lower()
    locationString = f"a {categoryName}"
    tone = ""
    if sentiment == "positive":
        sentiment = "great"
        inspire_sentence = "This photo is sure to delight and inspire anyone who sees it"
        if genre == "restaurant":
            sentiment = "delicious"
            tone = "delicious, well-decorated"
        elif genre == "hotel":
            sentiment = "great"
            tone = "clean"
        elif genre == "activity":
            inspire_sentence = "This photo is sure to inspire anyone who sees it to do the same activity."
            sentiment = "fun"
            tone = "fun, happy athmosphere"
    elif sentiment == "negative":
        inspire_sentence = "This photo is likely to disappoint and demotivate anyone who sees it"
        sentiment = "dissappointing"
        if genre == "restaurant":
            tone = "bad service, dirty, bad decoration"
        elif genre == "hotel":
            tone = "dirty, uncomfortable"
        elif genre == "activity":
            tone = "boring, lame"
    else:
        inspire_sentence = ""
        sentiment = "average"

    if genre == "restaurant":
       # PROMPT = f"A makro of {sentiment} food at a {categoryName}, {tone}, {inspire_sentence}, Sigma 105mm F2.8, close-up, hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Tripadvisor.com in 2022"
        PROMPT =  f"A photo of {sentiment} food at a {categoryName}. {inspire_sentence}. The shot is taken with a Canon EOS 5D Mark IV camera and a 40mm f/1.8 lens, with settings of f/5.6, ISO 100, and a shutter speed of 1/200 sec. Close-up, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2,  on Tripadvisor.com 2022"
    elif genre == "hotel":
        PROMPT = f"A photo of {sentiment} stay at a {categoryName}, {inspire_sentence}. Sigma lens, wide shot, full shot. hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Bookings.com in 2022"
    elif genre == "activity":
        PROMPT = f"A photo of a {categoryName} taken by a photograph visitor. The shot is taken with a Canon EOS 5D Mark IV camera and a 40mm f/1.8 lens, hyper detailed, Super-Resolution, UHD, DTM, HDR, 8K --ar 4:3 --q 2, on Tripadvisor.com in 2022"

    print(PROMPT)
    return PROMPT

prompt = generate_dalle_prompt(df_fake.iloc[599])
response = openai.Image.create(prompt=prompt, n=1, size="256x256")
print(response["data"][0]["url"])


In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')
print(df_fake.shape)

In [ ]:
df_fake = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')

df_fake['fake_reviewImageUrls/0'] = ""
print(df_fake.shape)

for index, row in df_fake.iterrows():
    if  index < 449:
        print("Schon berechnet (auf anderem Account): ", index)
        continue
    print("Generating Promt for row: ", index)
    prompt = generate_dalle_prompt(row)
    df_fake.at[index, 'dalle_prompt'] = prompt
    print("Generated Image for index: ", index)
    response = openai.Image.create(prompt=prompt, n=1, size="256x256")
    df_fake.at[index, 'fake_reviewImageUrls/0'] = response["data"][0]["url"]
    response_url = response["data"][0]["url"]
    print(response["data"][0]["url"])
    df_fake.to_csv('fake_base_gpt3_v23314_sent_dalle.csv', index=False)
    print("Saved Image for index: ", index)
    print(f"_______________________NOW START DOWNLOADING FOR INDEX {index} ____________________________________________________________________")

    reviewId = row['reviewId']
    reviewIdF = row['reviewIdF']
    print(f"Downloading Image for index {index}, reviewId {reviewId}, reviewIdF {reviewIdF}...")
    print(row['fake_reviewImageUrls/0'])
    response = requests.get(response_url)
    img = Image.open(BytesIO(response.content))
    print(img.mode)
    print(img.size)
    img.save(f'fake_image_url_0/{index}_{reviewId}_fake_reviewImageUrls_0.png')
    print("Saved Image for index: ", index)
    print("_______________________________DONE___________________________________________________________")

print("Fertig mit Erzeugung von Bildern mit API über Dalle 2.")

Eveentuell die erzeugten Bilder umbennen, falls System was aus Filenamen lernt bzw es die Filenamen nicht mehr passen wegen dem zusammengeführten Index später

...rename a file from folder fake_image_url_0


In [ ]:
import os
import glob

for filename in glob.glob('fake_image_url_0/*.png'):
    print(filename)
    # rename the files that the first 3 number 161_ are removed
    os.rename(filename, filename[4:])

In [3]:
pd.set_option('display.max_colwidth', None)
df_fake = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')
df_fake['githubUrls/0'] = ""
print(df_fake.shape)

for index, row in df_fake.iterrows():
    print("Generating Github Url for row: ", index)
    github_path = f"https://raw.githubusercontent.com/MichaelSeitz98/seminararbeit_review_detection/main/02_Images/fake_image_url_0/{index}_{row['reviewId']}_fake_reviewImageUrls_0.png"
    df_fake.at[index, 'githubUrls/0'] = github_path
    print(f"Generated {github_path} Github Url for index: ", index)

print("Fertig mit Erzeugung von Github Urls.")
print(df_fake["githubUrls/0"])

df_fake.to_csv('fake_base_gpt3_v23314_sent_dalle.csv', index=False)

(705, 55)
Generating Github Url for row:  0
Generated https://raw.githubusercontent.com/MichaelSeitz98/seminararbeit_review_detection/main/02_Images/fake_image_url_0/0_ChZDSUhNMG9nS0VJQ0FnSURJanBQb1pBEAE_fake_reviewImageUrls_0.png Github Url for index:  0
Generating Github Url for row:  1
Generated https://raw.githubusercontent.com/MichaelSeitz98/seminararbeit_review_detection/main/02_Images/fake_image_url_0/1_ChdDSUhNMG9nS0VJQ0FnSUNZLWRHY3VRRRAB_fake_reviewImageUrls_0.png Github Url for index:  1
Generating Github Url for row:  2
Generated https://raw.githubusercontent.com/MichaelSeitz98/seminararbeit_review_detection/main/02_Images/fake_image_url_0/2_ChdDSUhNMG9nS0VJQ0FnSUNtc04zQXNRRRAB_fake_reviewImageUrls_0.png Github Url for index:  2
Generating Github Url for row:  3
Generated https://raw.githubusercontent.com/MichaelSeitz98/seminararbeit_review_detection/main/02_Images/fake_image_url_0/3_ChZDSUhNMG9nS0VJQ0FnSUNvb05hU1pREAE_fake_reviewImageUrls_0.png Github Url for index:  3
Gene

In [23]:
df_fake = pd.read_csv('fake_base_gpt3_v23314_sent_dalle.csv')
print(df_fake.columns)
df_fake.to_excel('fake_base_gpt3_v23314_sent_dalle.xlsx', index=False)

Index(['stars', 'publishedAtDate', 'name', 'text', 'title', 'subTitle', 'description', 'price', 'totalScore', 'likesCount', 'isLocalGuide', 'reviewId', 'categoryName', 'reviewImageUrls/0', 'reviewImageUrls/1', 'reviewImageUrls/2', 'reviewImageUrls/3', 'reviewUrl', 'reviewerId', 'reviewerNumberOfReviews', 'reviewerPhotoUrl', 'reviewerUrl', 'reviewsCount', 'scrapedAt', 'state', 'street', 'temporarilyClosed', 'url', 'website', 'categories/0', 'categories/1', 'categories/2', 'categoryName.1', 'genre', 'keywords', 'keywords_only', 'sentiment', 'sent_score_0', 'label', 'placeId', 'text_length', 'prompt_v2', 'gpt3_v2', 'prompt_v3', 'gpt3_v3', 'gpt3_v3.1', 'prompt_v4', 'gpt3_v4', 'sent_v2', 'sent_v3', 'sent_v3.1', 'sent_v4', 'reviewIdF', 'dalle_prompt', 'githubUrls/0'], dtype='object')


## Text (OUTDATED) ChatGPT3.5 Turbo Verworfen

In [ ]:
class chatGPT: 
    def __init__(self, api_key, rolle):
        openai.api_key = api_key
        self.dialog = [{"role":"system", "content":rolle}]

    def question(self, question):
        self.dialog.append({"role":"user", "content":question})
        ergebnis = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=self.dialog)
        antwort = ergebnis["choices"][0].message.content
        self.dialog.append({"role":"assistant", "content":antwort})
        return antwort

In [ ]:
role = "In stlye of a google user that writes a review about a place he or she visited."
prompt = "Write a Google Maps review about a bowling alley with max 200 tokens."
chatGPT(api_key, role).question(prompt)
print(chatGPT(api_key, role).question(prompt))

In [ ]:
def get_keywords(text):
    keywords = model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=5)
    keywords = [keyword[0] for keyword in keywords]
    return keywords

output_string= "I recently visited the Italian Restaurant and overall had a great experience. The atmosphere was cozy and inviting with a classic Italian vibe. The staff was friendly and attentive, making sure we had everything we needed throughout our meal.The menu had a great selection of traditional Italian dishes and some unique options as well. I tried the gnocchi and it was delicious, perfectly cooked with a flavorful sauce. My friend had the lasagna and said it was some of the best she's ever had. The only downside was the wait time for our table, as we had to wait about 30 minutes even though we had reservations. However, the food and service made up for it. Overall, I would recommend the Italian Restaurant to anyone who loves Italian cuisine and wants to have a nice, cozy dinner out with friends or family."
keywords = get_keywords(output_string)
